# Step 0: Import thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# =================================================================
# Step 1: ĐỌC VÀ HỢP NHẤT DỮ LIỆU THÔ (Bắt buộc)
# =================================================================
# Đọc lại dữ liệu thô (để đảm bảo tính minh bạch, độc lập)

In [2]:
try:
    # Giả sử file train.csv và test.csv nằm cùng thư mục với notebook
    train_df = pd.read_csv('../data/train.csv')
    test_df = pd.read_csv('../data/test.csv')
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file CSV. Hãy đảm bảo chúng ở đúng vị trí.")

Y_train = train_df['Survived']
train_df = train_df.drop('Survived', axis=1)
data = pd.concat([train_df, test_df], ignore_index=True)

# =================================================================
# BƯỚC 2: KỸ THUẬT ĐẶC TRƯNG VÀ XỬ LÝ THIẾU GIÁ TRỊ (Logic từ EDA)
# =================================================================

# 2.1. Trích xuất và Gom nhóm Title (Tận dụng phân tích Name)

In [ ]:
data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
title_replacements = {
    'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs',
    'Lady': 'Royalty', 'Countess': 'Royalty', 'Dona': 'Royalty',
    'Capt': 'Military', 'Col': 'Military', 'Major': 'Military', 'Dr': 'Rare',
    'Rev': 'Rare', 'Don': 'Rare', 'Sir': 'Royalty', 'Jonkheer': 'Rare'
}
data['Title'] = data['Title'].replace(title_replacements)
# Tách lại dữ liệu (tạo copy để tránh SettingWithCopyWarning)
train_processed = data.iloc[:len(Y_train), :].copy()
test_processed = data.iloc[len(Y_train):, :].copy()
train_processed['Survived'] = Y_train.values

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\pikal\AppData\Local\Temp\ipykernel_17960\1602724743.py:1: SyntaxWarning: invalid escape sequence '\.'
  data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


# 2.2. Điền Age CÓ NGỮ CẢNH (Tận dụng phân tích Title)

In [4]:
# Điền Age: Dùng Trung vị (Median) của Title tương ứng
data['Age'] = data.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))

# 2.3. Tạo Family Size và Is Alone

In [5]:
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
data['IsAlone'] = (data['FamilySize'] == 1).astype(int)

# 2.4. Trích xuất Deck (Tận dụng phân tích Cabin)

In [6]:
data['Deck'] = data['Cabin'].apply(lambda x: x[0] if pd.notna(x) else 'U')

# 2.5. Xử lý thiếu giá trị còn lại (Embarked, Fare)

In [7]:
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
data['Fare'] = data['Fare'].fillna(data[data['Pclass'] == 3]['Fare'].median())

# =================================================================
# BƯỚC 3: MÃ HÓA VÀ CHUẨN HÓA (Tối ưu cho KNN và SVM)
# =================================================================

# 3.1. Loại bỏ các Cột không cần thiết

In [8]:
data = data.drop(['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1)

# 3.2. Mã hóa One-Hot Encoding

In [9]:
data['Pclass'] = data['Pclass'].astype('category') 
categorical_features = ['Sex', 'Embarked', 'Pclass', 'Title', 'Deck']
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

# 3.3. Chuẩn hóa Đặc trưng Số (Rất quan trọng cho KNN và SVM)

In [10]:
scaler = StandardScaler()
numerical_cols = ['Age', 'Fare']
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# =================================================================
# BƯỚC 4: TÁCH VÀ LƯU FILE OUTPUT TỐI ƯU
# =================================================================

In [11]:
# Tách lại dữ liệu
train_processed = data.iloc[:len(Y_train), :]
test_processed = data.iloc[len(Y_train):, :]
train_processed['Survived'] = Y_train.values

C:\Users\pikal\AppData\Local\Temp\ipykernel_17960\696499151.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_processed['Survived'] = Y_train.values


# LƯU KẾT QUẢ CUỐI CÙNG (File Input cho các mô hình)

In [12]:
train_processed.to_csv('../data/titanic_FINAL_INPUT_train.csv', index=False)
test_processed.to_csv('../data/titanic_FINAL_INPUT_test.csv', index=False)

In [13]:
print("✅ Quy trình Tiền xử lý Tối ưu (dựa trên EDA) đã hoàn tất và dữ liệu đã được lưu!")
print("Sử dụng các file 'titanic_FINAL_INPUT_...csv' cho bước huấn luyện mô hình tiếp theo.")

✅ Quy trình Tiền xử lý Tối ưu (dựa trên EDA) đã hoàn tất và dữ liệu đã được lưu!
Sử dụng các file 'titanic_FINAL_INPUT_...csv' cho bước huấn luyện mô hình tiếp theo.
